<p><img alt="Colaboratory logo" height="50px" src="http://ingeotec.mx/~mgraffg/ingeotec-bg.png" align="left" hspace="10px" vspace="0px"></p>

<h1>$\mu$TC's Quickstart Guide</h1>

### http://ingeotec.mx

### http://github.com/ingeotec

## Install $\mu$TC

In [ ]:
!pip install microtc

# First model

## Reading a dataset

In [ ]:
!wget https://raw.githubusercontent.com/INGEOTEC/EvoMSA/master/EvoMSA/tests/tweets.json

In [ ]:
from microtc.utils import tweet_iterator
D = list(tweet_iterator("tweets.json"))

In [ ]:
D[0]

## Bag of Words

In [ ]:
from microtc.textmodel import TextModel
tm = TextModel(token_list=[-1, -2, (2, 1), 4]).fit(D)

In [ ]:
tm["hola mundo"]

In [ ]:
tm.tokenize("hello world")

## Text Classifier

In [ ]:
from sklearn.svm import LinearSVC
X = tm.transform(D)
m = LinearSVC().fit(X, [x["klass"] for x in D])

In [ ]:
m.predict(tm.transform(["buenos dias"]))

# Performance

In [ ]:
from sklearn import model_selection
from sklearn.metrics import f1_score
import numpy as np
from tqdm import tqdm
kfold = model_selection.StratifiedKFold(n_splits=5, shuffle=True)

In [ ]:
P = []
X = np.empty((len(D), 1))
for tr, vs in tqdm(kfold.split(X, [x["klass"] for x in D]), total=5):
  D_tr = [D[x] for x in tr]
  D_vs = [D[x] for x in vs]
  tm = TextModel(token_list=[-1, -2, 3]).fit(D_tr)
  X = tm.transform(D_tr)
  m = LinearSVC().fit(X, [x["klass"] for x in D_tr])
  hy = m.predict(tm.transform(D_vs))
  _ = f1_score([x["klass"] for x in D_vs], hy, average=None)
  P.append(_)
np.mean(P, axis=0)

In [ ]:
array([0.56117472, 0.        , 0.42189925, 0.72769939])

In [ ]:
P